#### 예제1. 플러그형 할인 전략을 가진 Order 클래스 구현

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [2]:
jack = Customer('Park', 100)
jack

Customer(name='Park', fidelity=100)

In [3]:
class LineItem:
    """구매할 물품/갯수 생성해서 총 가격을 반환"""
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [4]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
        
    def total(self):
        """__total 속성이 없으면 전체 계산할 값을 계산"""
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        """할인금액 차감"""
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self) # self = Order 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [5]:
class Promotion(ABC): # strategy abstract base class
    """할인 혜택 클래스들의 형태를 선언"""
    
    @abstractmethod # 이 클래스를 상속하는 클래스는 반드시 이 메서드를 선언해야함선언해야
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

In [6]:
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

#### 예제2. 여러 프로모션 할인을 적용해서 Order 클래스를 사용하는 예

In [7]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [8]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [9]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [10]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [12]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [13]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)] # 물품 종류가 10개

In [14]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [15]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

#### 예제3. 할인 전략을 함수로 구현한 Order 클래스

In [21]:
class Order: # Context
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion # 할인 객체
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # self = Order 객체
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [22]:
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [23]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [24]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [25]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [26]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

#### 예제4. 함수 리스트를 반복해서 최대 할인액을 찾아내는 best_promo() 함수

In [27]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
# 함수들로 구현된 전략들의 리스트

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

#### 예제5. 모든 할인을 적용해서 가장 큰 값을 반환하는 best_promo() 함수

In [28]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [29]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [30]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [31]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__vsc_ipynb_file__', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', 'namedtuple', 'Customer', '_i2', 'jack', '_2', '_i3', 'LineItem', '_i4', 'Order', '_i5', 'Promotion', '_i6', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i7', 'joe', 'ann', 'cart', '_i8', '_8', '_i9', '_9', '_i10', 'banana_cart', '_i11', '_11', '_i12', '_12', '_i13', 'long_order', '_i14', '_14', '_i15', '_15', '_i16', '_i17', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i18', '_18', '_i19', '_i20', '_20', '_i21', '_i22', '_i23', '_23', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27', 'promos', 'best_promo', '_i28', '_28', '_i29', '_29', '_i30', '_30', '_i31'])

In [32]:
globals()['best_promo']

<function __main__.best_promo(order)>

#### 예제6. 모듈 전역 네임스페이스를 내부 조사해서 만든 promos 리스트

In [33]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo'] # 재귀호출을 피하기 위함
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

#### 예제1. 일반적으로 Decorator는 함수를 다른 함수로 대체한다.

In [34]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [35]:
@deco
def target():
    print('running target()')

target() # 실제로는 inner()를 실행

running inner()


In [36]:
target # inner()를 가리킴

<function __main__.deco.<locals>.inner()>

#### 예제2. Decorator를 실행하는 시점 예제

In [37]:
registry = [] # 여기에 decorated function 추가

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
    
if __name__ == '__main__':
    main()
    

running register(<function f1 at 0x107a9c040>)
running register(<function f2 at 0x107c31b80>)
running main()
registry -> [<function f1 at 0x107a9c040>, <function f2 at 0x107c31b80>]
running f1()
running f2()
running f3()


#### 예제3. Promotion decorator로 채운 promos list

In [39]:
promos = []

def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

#### 예제4. 지역 및 전역 변수를 읽는 함수

In [40]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [41]:
b = 6
f1(3)

3
6


#### 예제5. 함수 본체 안에서 값을 할당하기 때문에 지역 변수가 되는 b

In [42]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

#### 예제6. global 선언 후 변화

In [43]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [45]:
f3(3)

3
9


In [46]:
b = 30
f3(3)

3
30


#### 예제7. 이동 평균을 계산하는 클래스

In [47]:
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [48]:
avg = Averager()
avg(10)

10.0

In [49]:
avg(11)

10.5

In [50]:
avg(12)

11.0

#### 예제8. 이동 평균을 계산하는 high-order function

In [51]:
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [52]:
avg = make_averager()
avg(10)

10.0

In [53]:
avg(11)

10.5

In [54]:
avg(12)

11.0

#### 예제9. make_averager()로 생성한 함수 조사하기

In [55]:
avg.__code__.co_varnames

('new_value', 'total')

In [56]:
avg.__code__.co_freevars

('series',)

In [57]:
avg.__closure__

(<cell at 0x10773da60: list object at 0x107d1c8c0>,)

In [58]:
avg.__closure__[0].cell_contents

[10, 11, 12]

#### 예제10. 전체 이력을 유지하지 않고 이동 평균을 잘못 계산하는 잘못된 high-order function

In [59]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

In [60]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

#### 예제11. 전체 이력을 유지하지 않고 이동 평균 계산하기 (nonlocal로 수정)

In [61]:
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [62]:
avg = make_averager()
avg(10)

10.0

#### 예제12. 함수의 실행 시간을 출력하는 간단한 decorator

In [65]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

#### 예제13. clock decorator 사용하기

In [66]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12804296s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000079s] factorial(1) -> 1
[0.00002275s] factorial(2) -> 2
[0.00004029s] factorial(3) -> 6
[0.00007029s] factorial(4) -> 24
[0.00009458s] factorial(5) -> 120
[0.00012350s] factorial(6) -> 720
6! = 720


In [67]:
factorial.__name__

'clocked'

#### 예제14. 개선된 clock decorator

In [68]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(paris))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [69]:
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12333129s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000121s] factorial(1) -> 1
[0.00003321s] factorial(2) -> 2
[0.00005529s] factorial(3) -> 6
[0.00007846s] factorial(4) -> 24
[0.00010162s] factorial(5) -> 120
[0.00012508s] factorial(6) -> 720
6! = 720


In [70]:
factorial.__name__

'factorial'